<a href="https://colab.research.google.com/github/leonesso/pre-processamento-cerveja-consumo/blob/main/Processo_de_modelagem_com_aprendizado_de_maquina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Processo de modelagem com Machine Learning

<img src = 'https://frnsys.com/assets/hosny/disparity.gif' width =600>


Neste notebook vamos apresentar conceitos e códigos referentes a um processo de modelagem do inicio ao fim, testando o modelo desenvolvido.

Esse processo utiliza das fases do CRISP-DM e serve como uma referência para outras modelagens.


## Processo básico de modelagem

É composto por 6 passos:

* Fazer a pergunta certa
* Preparar os dados
* Selecionar o algoritmo
* Treinar o modelo
* Testar o modelo
* Implementar o modelo

### Preparando os dados


Antes de iniciar a preparação dos dados necessitamos importar um conjunto de bibliotecas referente a criação de estruturas de dados, entendimento numérico dos tipos de dados além de bibliotecas gráficas.


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('seaborn')

Agora vamos carregar uma base de dados sobre consumo de cerveja e analisar o quanto o clima impacta nisto

<img src = 'https://media.giphy.com/media/42dsvcMDP3diU/giphy.gif' width=500>

In [ ]:
df_cerveja = pd.read_excel("beer_consumption.xlsx")
df_cerveja.head(10)

Vamos ver as estatísticas descritivas desta base

In [ ]:
df_cerveja.describe()

In [ ]:
df_cerveja.count()

In [ ]:
df_cerveja.corr()

In [ ]:
sns.heatmap(df_cerveja.corr())
#sns.heatmap(df_cerveja.corr(), cmap='viridis')
#sns.heatmap(df_cerveja.corr(), cmap='viridis', annot=True)

Não foram encontrados colunas correlacionadas o suficiente para removê-las.

### Compreendendo relações e correlações de variáveis

sns.lmplot()

In [ ]:
sns.lmplot(data=df_cerveja, x="beer_consumption", y="temp_avg",
          scatter_kws={"marker":"x", "color":"blue"},
          line_kws={"linewidth":1, "color": "orange"})

In [ ]:
sns.lmplot(data=df_cerveja, x="beer_consumption", y="temp_max",
          scatter_kws={"marker":"x", "color":"gray"},
          line_kws={"linewidth":1, "color": "orange"})

In [ ]:
sns.lmplot(data=df_cerveja, x="precip", y="beer_consumption",
          scatter_kws={"marker":"x", "color":"green"},
          line_kws={"linewidth":1, "color": "orange"})

### Mapear dados em label para numéricos

In [ ]:
df_cerveja.head()

In [ ]:
# Categorização
class_weekend = {True: 1, False: 0}
df_cerveja["weekend"] = df_cerveja["weekend"].map(class_weekend)

In [ ]:
df_cerveja.head(5)

### Busca por dados nulos e inválidos

In [ ]:
df_cerveja.isnull().any()

In [ ]:
df_cerveja[df_cerveja["temp_avg"].isnull()]

In [ ]:
df_cerveja[df_cerveja["temp_min"].isnull()]

In [ ]:
df_cerveja[df_cerveja["temp_max"].isnull()]

In [ ]:
df_cerveja[df_cerveja["weekend"].isnull()]

Temperatura média pode ser calculada entre a temperatura máxima e mínima

In [ ]:
df_cerveja_temp_avg_null = df_cerveja[df_cerveja["temp_avg"].isnull()].copy()
df_cerveja_temp_avg_null

In [ ]:
df_cerveja_temp_avg_null["temp_avg"] = (df_cerveja["temp_max"] + df_cerveja["temp_min"])/2
df_cerveja_temp_avg_null

In [ ]:
# outra forma de retornar a média da coluna
df_cerveja["temp_avg"].mean()

In [ ]:
# df_cerveja["temp_avg"] = df_cerveja["temp_avg"].replace(np.nan,(df_cerveja["temp_max"] + df_cerveja["temp_min"])/2)
df_cerveja["temp_avg"] = df_cerveja["temp_avg"].replace(np.nan,df_cerveja["temp_avg"].mean())

In [ ]:
df_cerveja.loc[[168,181,309,314]]

In [ ]:
df_cerveja["temp_min"] = df_cerveja["temp_min"].replace(np.nan,df_cerveja["temp_min"].mean())

In [ ]:
df_cerveja.loc[[7,116]]

In [ ]:
df_cerveja.loc[[98, 165, 237]]

In [ ]:
df_cerveja["temp_max"] = df_cerveja["temp_max"].replace(np.nan, df_cerveja["temp_max"].mean())

In [ ]:
df_cerveja.loc[[98, 165, 237]]

In [ ]:
# calcular a moda do final de semana
df_cerveja["weekend"].mode()

In [ ]:
df_cerveja["weekend"] = df_cerveja["weekend"].replace(np.nan, df_cerveja["weekend"].mode()[0])

In [ ]:
df_cerveja.loc[[21,27]]

In [ ]:
df_cerveja.isnull().any()

**Verifcar ocorrência de números iguais a 0, que seria inválido**

In [ ]:
(df_cerveja == 0).any()

In [ ]:
df_cerveja[df_cerveja["temp_avg"] == 0]

In [ ]:
df_cerveja[df_cerveja["temp_min"] == 0]

In [ ]:
df_cerveja[df_cerveja["temp_max"] == 0]

In [ ]:
df_cerveja.shape

In [ ]:
df_cerveja[df_cerveja["precip"] == 0].head(10)

In [ ]:
df_cerveja[df_cerveja["weekend"] == 0].head(10)
#df_cerveja[df_cerveja["weekend"] == 0].head(10).sort_values(by='data', ascending=False)

Precipitação e Weekend podem ser igual a 0.

Utilizar mesma estratégia anterior para os demais atributos.

In [ ]:
df_cerveja["temp_avg"] = df_cerveja["temp_avg"].replace(0, df_cerveja["temp_avg"].mean())

In [ ]:
df_cerveja["temp_min"] = df_cerveja["temp_min"].replace(0, df_cerveja["temp_min"].mean())

In [ ]:
df_cerveja["temp_max"] = df_cerveja["temp_max"].replace(0, df_cerveja["temp_max"].mean())

In [ ]:
(df_cerveja == 0).any()

### Separando dados de treinamento e teste

In [ ]:
from sklearn.model_selection import train_test_split

feature_col_names = ['temp_max', 'precip', 'weekend']
predicted_class_names = ['beer_consumption']

X = df_cerveja[feature_col_names].values
y = df_cerveja[predicted_class_names].values
split_test_size = 0.30

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split_test_size, random_state=42)

In [ ]:
print("{0:0.2f}% in training set".format((len(X_train)/len(df_cerveja.index)) * 100))
print("{0:0.2f}% in test set".format((len(X_test)/len(df_cerveja.index)) * 100))

In [ ]:
from sklearn import linear_model

lr_model = linear_model.LinearRegression()
lr_model.fit(X_train, y_train.ravel())

Testando valores par verificar a performance

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = lr_model.predict(X_test)

print('R2 score: %.2f' % r2_score(y_test, y_pred))
print('R2 score: %.2f' % lr_model.score(X_test, y_test))

R2 score está na faixa esperada de >= 70%

Simulação de um caso onde se espera maior consumo, clima quente, 35 graus C, sem chuva e é final de semana.

In [ ]:
predict_value = [[35, 0, 1]]
lr_model.predict(predict_value)

Simulação de um caso onde se espera um consumo menor que o de cima, clima quente, 35 graus C, sem chuva mas em dia de semana.

In [ ]:
predict_value = [[35, 0, 0]]
lr_model.predict(predict_value)

Simulação de um caso onde se espera um consumo menor que o de cima, clima quente, 35 graus C, com chuva e em dia de semana.

In [ ]:
predict_value = [[35, 20, 0]]
lr_model.predict(predict_value)

Pior cenário, baixo consumo, frio (10 graus C), chovendo e em dia de semana.

In [ ]:
predict_value = [[10, 20, 0]]
lr_model.predict(predict_value)

### Salvando modelo

In [ ]:
#from sklearn.externals import joblib
import joblib
joblib.dump(lr_model, 'lr_model.pkl')

In [ ]:
lr_carregar_modelo = joblib.load('lr_model.pkl')

In [ ]:
novos_valores = [[10, 20, 0]]
lr_carregar_modelo.predict(novos_valores)